<h1>LangChain Beispiel</h1>

Wieso sollte man LangChain nutzen und nicht einfach ChatGPT? <br>
Oft will man ein eigenes LLM Model nutzen das mit den eigenen bestehenden Daten trainiert wurde. Das hat den Vorteil das keine sensitiven Daten nach außen dringen zu Webseiten, die ein Model bereitstellen und potenziell diese Daten die geschickt wurden speichert. Mit einem eigenen Model bleibt das Model Lokal auf dem PC oder im Unternehmensnetzwerk. Zudem haben Modelle die ChatGPT keinen direkten Zugriff auf unsere Daten.

Open-Source Modelle können auch einfach importiert und gespeichert werden, damit fällt das Haupttraining des Netzes weg (ggf. Fine-Tuning). Mit LangChain können solche Applikationen mit Modellen erstellt werden. Das Framework bietet verschiedenste Tools für den Umgang mit solchen Modellen.

Als Einstieg wollen wir ein Open-Source Modell nutzen und einfache Textabfragen abwickeln. Weitere Beispiele Folgen.

<i>Abb1</i>: ChatGPT nutzung. Kommunikation über Internet.

<img src="./data/img/1_lg.PNG" width=700 height=500>

*Um ChatGPT zu nutzen, muss ein OpenAI API Key bereitgestellt werden. Siehe OpenAI API <br>
*Kosten können anfallen.

Mehr über LangChain:

> https://python.langchain.com/v0.2/docs/introduction/ [Letzter Zugriff: 01.08.2024]

HuggingFace Modelle und Beispiele: <br>
> https://huggingface.co/docs/transformers/model_doc/gpt2 [Letzter Zugriff: 01.08.2024] 

Es gibt eine große Anzahl von Modellen, die wir nutzen können. Als Einstieg nutzen wir das GPT2 Model.<br>
Auf der Webseite von HuggingFace wo die Modelle aufgelistet sind, befinden sich für jedes Model Beschreibungen und Beispiele.

Es sollte genug Speicher für ein Model bereitgestellt werden. 

GPT2: <br>
> https://huggingface.co/docs/transformers/model_doc/gpt2 [Letzter Zugriff: 01.08.2024]

Dieses Model wurde mit den Daten aus 8 Millionen Webseiten trainiert. <u>Das Ziel</u>: die nächsten Wörter vorhersagen.

Jedes Model kann andere Ziele haben.

In [1]:
# Imports.
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
# Erstelle Model.
model       = AutoModelForCausalLM.from_pretrained("gpt2")  # Model, welches wir nutzen wollen. 
tokenizer   = AutoTokenizer.from_pretrained("gpt2")  # Tokenizer des Models => Bereite Eingabe vor. 
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

Jedes Model ist anders und braucht einen anderen Input. Je nach Model und Aufgabe wird ein Tokenizer erstellt der den Text (oder andere Inputs) vorverarbeitet, um diese dann dem Model zu übergeben. Ohne ein Tokenizer müsste man alle Vorverarbeitungsschritte selber durchführen.

In [3]:
tokenizer("How are you doing?")

{'input_ids': [2437, 389, 345, 1804, 30], 'attention_mask': [1, 1, 1, 1, 1]}

In [4]:
prompt = "The weather is sunny today, so I will need "

# return_tensors: Rückgabetyp, hier: PyTorch. 
encoded_input  = tokenizer(prompt, return_tensors="pt")
encoded_input

{'input_ids': tensor([[  464,  6193,   318, 27737,  1909,    11,   523,   314,   481,   761,
           220]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [5]:
input_ids      = encoded_input['input_ids']
attention_mask = encoded_input['attention_mask']

In [6]:
gen_tokens = model.generate(
    input_ids,  # Startpunkt für die Generierung.
    attention_mask=attention_mask,  # Ignoriere padding Tokens.  
    do_sample=True,   # Erlaubt Sampling:True: erlaubt kreative outputs. 
    temperature=0.5,  # Wie "kreativ" das Model sein soll => Randomness 
    max_length=25,    # Output Länge
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [7]:
gen_tokens

tensor([[  464,  6193,   318, 27737,  1909,    11,   523,   314,   481,   761,
           220,  1849,    64,  1310,  1643,   286,  6290,   284,   651,   284,
           262,  5093,   286,   262,  1748]])

In [8]:
# batch_decode: Token IDs zurück zu Wörtern. 
# - Weitere Wörter werden generiert.
gen_text = tokenizer.batch_decode(gen_tokens)[0]
print(gen_text)

The weather is sunny today, so I will need  a little bit of rain to get to the north of the city


Das war ein erstes einfaches Beispiel wie man ein Model nutzt. <br>

<h2>Promt Templates und Chains</h2>

Statt immer wieder den String zu ändern, können Templates verwendet werden, um Änderungen direkt im Text vorzunehmen.

Hier nutzen wir dasselbe Model.

In [9]:
from langchain_core.prompts.prompt import PromptTemplate

In [10]:
# Erstelle Template
my_template = PromptTemplate(
    input_variables= ['replace'],
    template=        "Today the weather is {replace}, so I will... "
)

# Teste Template
my_template.format(replace="sunny") 

'Today the weather is sunny, so I will... '

So können verschiedenen Elemente direkt im Text ersetzt werden. Wie der Name schon sagt, wird ein Template erstellt, wo bestimmte Lücken gefüllt werden können.

<i>Abb2</i>: Prompt Template, fülle die Lücken.

<img src="./data/img/2_lg.PNG" width=400 height=200>

In [13]:
# Weiteres Beispiel # 

template = "Tell me {x-number} of facts about the topic {topic}"  # Text Template.
promt_template = PromptTemplate.from_template(template)  # Erstelle Template.

promt =  promt_template.invoke({"x-number": 5, "topic": "dog" })
promt

StringPromptValue(text='Tell me 5 of facts about the topic dog')

Für den Anfang reicht es.

Dann gibt es noch Chains mit dem Operationen verkettet werden können.

<i>Abb1:</i> [coming soon]

<img src="">